In [ ]:
# Importing required packages
import glob
from operator import contains
import os, json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import re
pd.options.display.float_format = '{:.15f}'.format

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/Mercy

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/Mercy


In [ ]:
# Importing the csv file
# data = pd.read_csv('C:/Users/CJ/final.csv')
# data = pd.read_csv('final.csv')
data = pd.read_csv('final_1416.csv')
# data = pd.read_csv('D:/clair/notes/Sem4/Capstone/dataset/export/feb_26_v_1.0/VeReMi_54000_57600_2019-11-28_10_42_1.csv')
data.head()

,type,rcvTime,sendTime,sender,senderPseudo,messageID,file_name,density_level,pos_x,pos_y,...,acl_noise_y,acl_noise_z,hed_x,hed_y,hed_z,hed_noise_x,hed_noise_y,hed_noise_z,attack?,receiver_id
0,3,57511.049951916953432,57511.049951916953432,9891.000000000000000,1098915.000000000000000,19655176.000000000000000,traceJSON-10005-10003-A0-57511-15,2,137.286902555145161,983.827257071713575,...,0.000021258383547,0.000000000000000,-0.036686450374153,0.999326825597584,0.000000000000000,4.249188192325861,0.982232769632962,0.000000000000000,1,10005
1,3,57511.146510437574761,57511.146510437574761,9951.000000000000000,1099515.000000000000000,19655429.000000000000000,traceJSON-10005-10003-A0-57511-15,2,153.904668561966304,899.505911646706977,...,0.000000000000013,0.000000000000000,0.038287039242177,-0.999266782509089,0.000000000000000,9.636084890334937,12.342153049505139,0.000000000000000,1,10005
2,3,57511.162338829126384,57511.162338829126384,10011.000000000000000,10100115.000000000000000,19655584.000000000000000,traceJSON-10005-10003-A0-57511-15,2,142.981000552607952,1038.129049878944215,...,0.000424009628911,0.000000000000000,0.073948678480006,-0.997262048285735,0.000000000000000,7.179251774207310,7.627622845974860,0.000000000000000,1,10005
3,3,57511.590730855474249,57511.590730855474249,9957.000000000000000,1099575.000000000000000,19656248.000000000000000,traceJSON-10005-10003-A0-57511-15,2,149.931080349069163,899.035172728899624,...,0.000000000062721,0.000000000000000,0.016440994151280,-0.999864837721238,0.000000000000000,16.013310920410738,21.388849080932467,0.000000000000000,0,10005
4,3,57511.641619857240585,57511.641619857240585,9963.000000000000000,1099635.000000000000000,19656466.000000000000000,traceJSON-10005-10003-A0-57511-15,2,152.753405416050725,906.773024725052210,...,0.000000000000000,0.000000000000000,0.047863865685627,-0.998853868371959,0.000000000000000,6.510268774099048,7.224761247950584,0.000000000000000,0,10005


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 820097 entries, 0 to 820096
Data columns (total 34 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   type           820097 non-null  int64  
 1   rcvTime        820097 non-null  float64
 2   sendTime       820097 non-null  float64
 3   sender         820097 non-null  float64
 4   senderPseudo   820097 non-null  float64
 5   messageID      820097 non-null  float64
 6   file_name      820097 non-null  object 
 7   density_level  820097 non-null  int64  
 8   pos_x          820097 non-null  float64
 9   pos_y          820097 non-null  float64
 10  pos_z          820097 non-null  float64
 11  pos_noise_x    820097 non-null  float64
 12  pos_noise_y    820097 non-null  float64
 13  pos_noise_z    820097 non-null  float64
 14  spd_x          820097 non-null  float64
 15  spd_y          820097 non-null  float64
 16  spd_z          820097 non-null  float64
 17  spd_noise_x    820097 non-nul

In [ ]:
#moving the file name column to the last
colm = list(data.columns.values)
colm.pop(colm.index('file_name'))
data = data[colm+['file_name']]

In [ ]:
data.head()

,type,rcvTime,sendTime,sender,senderPseudo,messageID,density_level,pos_x,pos_y,pos_z,...,acl_noise_z,hed_x,hed_y,hed_z,hed_noise_x,hed_noise_y,hed_noise_z,attack?,receiver_id,file_name
0,3,57511.049951916953432,57511.049951916953432,9891.000000000000000,1098915.000000000000000,19655176.000000000000000,2,137.286902555145161,983.827257071713575,0.000000000000000,...,0.000000000000000,-0.036686450374153,0.999326825597584,0.000000000000000,4.249188192325861,0.982232769632962,0.000000000000000,1,10005,traceJSON-10005-10003-A0-57511-15
1,3,57511.146510437574761,57511.146510437574761,9951.000000000000000,1099515.000000000000000,19655429.000000000000000,2,153.904668561966304,899.505911646706977,0.000000000000000,...,0.000000000000000,0.038287039242177,-0.999266782509089,0.000000000000000,9.636084890334937,12.342153049505139,0.000000000000000,1,10005,traceJSON-10005-10003-A0-57511-15
2,3,57511.162338829126384,57511.162338829126384,10011.000000000000000,10100115.000000000000000,19655584.000000000000000,2,142.981000552607952,1038.129049878944215,0.000000000000000,...,0.000000000000000,0.073948678480006,-0.997262048285735,0.000000000000000,7.179251774207310,7.627622845974860,0.000000000000000,1,10005,traceJSON-10005-10003-A0-57511-15
3,3,57511.590730855474249,57511.590730855474249,9957.000000000000000,1099575.000000000000000,19656248.000000000000000,2,149.931080349069163,899.035172728899624,0.000000000000000,...,0.000000000000000,0.016440994151280,-0.999864837721238,0.000000000000000,16.013310920410738,21.388849080932467,0.000000000000000,0,10005,traceJSON-10005-10003-A0-57511-15
4,3,57511.641619857240585,57511.641619857240585,9963.000000000000000,1099635.000000000000000,19656466.000000000000000,2,152.753405416050725,906.773024725052210,0.000000000000000,...,0.000000000000000,0.047863865685627,-0.998853868371959,0.000000000000000,6.510268774099048,7.224761247950584,0.000000000000000,0,10005,traceJSON-10005-10003-A0-57511-15


In [ ]:
data.shape

(820097, 34)

In [ ]:
data.columns

Index(['type', 'rcvTime', 'sendTime', 'sender', 'senderPseudo', 'messageID',
       'density_level', 'pos_x', 'pos_y', 'pos_z', 'pos_noise_x',
       'pos_noise_y', 'pos_noise_z', 'spd_x', 'spd_y', 'spd_z', 'spd_noise_x',
       'spd_noise_y', 'spd_noise_z', 'acl_x', 'acl_y', 'acl_z', 'acl_noise_x',
       'acl_noise_y', 'acl_noise_z', 'hed_x', 'hed_y', 'hed_z', 'hed_noise_x',
       'hed_noise_y', 'hed_noise_z', 'attack?', 'receiver_id', 'file_name'],
      dtype='object')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 820097 entries, 0 to 820096
Data columns (total 34 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   type           820097 non-null  int64  
 1   rcvTime        820097 non-null  float64
 2   sendTime       820097 non-null  float64
 3   sender         820097 non-null  float64
 4   senderPseudo   820097 non-null  float64
 5   messageID      820097 non-null  float64
 6   density_level  820097 non-null  int64  
 7   pos_x          820097 non-null  float64
 8   pos_y          820097 non-null  float64
 9   pos_z          820097 non-null  float64
 10  pos_noise_x    820097 non-null  float64
 11  pos_noise_y    820097 non-null  float64
 12  pos_noise_z    820097 non-null  float64
 13  spd_x          820097 non-null  float64
 14  spd_y          820097 non-null  float64
 15  spd_z          820097 non-null  float64
 16  spd_noise_x    820097 non-null  float64
 17  spd_noise_y    820097 non-nul

In [ ]:
data.rename(columns = {'attack?':'attack'}, inplace = True)

In [ ]:
#Dropping the columns with noise

for i in data.columns:
    if 'noise' in i:
        data.drop(i, axis=1, inplace=True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 820097 entries, 0 to 820096
Data columns (total 22 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   type           820097 non-null  int64  
 1   rcvTime        820097 non-null  float64
 2   sendTime       820097 non-null  float64
 3   sender         820097 non-null  float64
 4   senderPseudo   820097 non-null  float64
 5   messageID      820097 non-null  float64
 6   density_level  820097 non-null  int64  
 7   pos_x          820097 non-null  float64
 8   pos_y          820097 non-null  float64
 9   pos_z          820097 non-null  float64
 10  spd_x          820097 non-null  float64
 11  spd_y          820097 non-null  float64
 12  spd_z          820097 non-null  float64
 13  acl_x          820097 non-null  float64
 14  acl_y          820097 non-null  float64
 15  acl_z          820097 non-null  float64
 16  hed_x          820097 non-null  float64
 17  hed_y          820097 non-nul

In [ ]:
# Checking if there is any missing values
data.isnull().sum()

type             0
rcvTime          0
sendTime         0
sender           0
senderPseudo     0
messageID        0
density_level    0
pos_x            0
pos_y            0
pos_z            0
spd_x            0
spd_y            0
spd_z            0
acl_x            0
acl_y            0
acl_z            0
hed_x            0
hed_y            0
hed_z            0
attack           0
receiver_id      0
file_name        0
dtype: int64

In [ ]:
# Checking correlation coeffecients of the variables.
data.corr()

,type,rcvTime,sendTime,sender,senderPseudo,messageID,density_level,pos_x,pos_y,pos_z,...,spd_y,spd_z,acl_x,acl_y,acl_z,hed_x,hed_y,hed_z,attack,receiver_id
type,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rcvTime,NaN,1.000000000000000,1.000000000000000,0.991190359027257,0.332383531192960,0.981938868170610,NaN,-0.015515565129284,-0.054679037679176,NaN,...,-0.015323880722068,NaN,-0.001321776695552,0.001102053848250,NaN,-0.012715235817638,0.003975628017103,NaN,-0.010610845112419,0.991190143857419
sendTime,NaN,1.000000000000000,1.000000000000000,0.991190359027257,0.332383531192960,0.981938868170610,NaN,-0.015515565129284,-0.054679037679176,NaN,...,-0.015323880722068,NaN,-0.001321776695552,0.001102053848250,NaN,-0.012715235817638,0.003975628017103,NaN,-0.010610845112419,0.991190143857419
sender,NaN,0.991190359027257,0.991190359027257,1.000000000000000,0.353547059459899,0.997256366007319,NaN,-0.020109011916664,-0.057349131484451,NaN,...,-0.009677792760889,NaN,-0.001823968843078,0.000589499966495,NaN,-0.011231973551745,0.001274708450362,NaN,-0.009724217030837,0.999067668168640
senderPseudo,NaN,0.332383531192960,0.332383531192960,0.353547059459899,1.000000000000000,0.351455133570445,NaN,-0.028114336080015,-0.027093650083626,NaN,...,0.002886852026981,NaN,-0.002761577857761,0.001385549081866,NaN,-0.012705653303939,-0.037876420754248,NaN,-0.006299526366066,0.352186664354456
messageID,NaN,0.981938868170610,0.981938868170610,0.997256366007319,0.351455133570445,1.000000000000000,NaN,-0.024717758973889,-0.054601583291376,NaN,...,-0.005023173557157,NaN,-0.001714075488471,0.000153806873852,NaN,-0.012034421967667,0.008525709342418,NaN,-0.009436781539420,0.997256736499987
density_level,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pos_x,NaN,-0.015515565129284,-0.015515565129284,-0.020109011916664,-0.028114336080015,-0.024717758973889,NaN,1.000000000000000,0.372677375533615,NaN,...,-0.074750228931128,NaN,-0.082842996087201,-0.030621052843420,NaN,-0.054851927249641,-0.092036022543302,NaN,-0.021627552551043,-0.020161378605513
pos_y,NaN,-0.054679037679176,-0.054679037679176,-0.057349131484451,-0.027093650083626,-0.054601583291376,NaN,0.372677375533615,1.000000000000000,NaN,...,-0.071619230453340,NaN,-0.012134974632444,-0.149201512631111,NaN,-0.019212280060194,-0.204881299176818,NaN,-0.012766575466645,-0.057085288679565
pos_z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Checking the different percentage of quantiles in the data & count of rows, mean, standard deviation, minimum value in column, maximum value in column.
data.describe(percentiles = [0,0.2,0.4,0.5,0.7,0.8,1])

,type,rcvTime,sendTime,sender,senderPseudo,messageID,density_level,pos_x,pos_y,pos_z,...,spd_y,spd_z,acl_x,acl_y,acl_z,hed_x,hed_y,hed_z,attack,receiver_id
count,820097.000000000000000,820097.000000000000000,820097.000000000000000,820097.000000000000000,820097.000000000000000,820097.000000000000000,820097.000000000000000,820097.000000000000000,820097.000000000000000,820097.000000000000000,...,820097.000000000000000,820097.000000000000000,820097.000000000000000,820097.000000000000000,820097.000000000000000,820097.000000000000000,820097.000000000000000,820097.000000000000000,820097.000000000000000,820097.000000000000000
mean,3.000000000000000,53361.914964955984033,53361.914964955984033,4668.517515610958071,998646.032816849765368,9741264.500440802425146,2.000000000000000,466.005537964691541,647.115351907733384,0.000000000000000,...,6.186811353363457,0.000000000000000,-0.000480762746271,-0.023169210227424,0.000000000000000,-0.103219741185455,0.047495566577841,0.000000000000000,0.302473975639467,4668.531123757311434
std,0.000000000000000,2030.832066308444837,2030.832066308444837,2755.659176036780536,634336.219720671768300,5518492.198153992183506,0.000000000000000,378.466030238798794,285.775630350415781,0.000000000000000,...,12.808933277606826,0.000000000000000,0.882635820342472,1.115253300678164,0.000000000000000,0.622126274364223,0.774628916680646,0.000000000000000,0.459329649559298,2755.658670892383270
min,3.000000000000000,50408.529003064497374,50408.529003064497374,9.000000000000000,1095.000000000000000,16936.000000000000000,2.000000000000000,24.773158019992341,24.326830246085336,0.000000000000000,...,-17.710941839353161,0.000000000000000,-4.499679603676096,-4.499568253197682,0.000000000000000,-0.999999999710040,-0.999999999997136,0.000000000000000,0.000000000000000,9.000000000000000
0%,3.000000000000000,50408.529003064497374,50408.529003064497374,9.000000000000000,1095.000000000000000,16936.000000000000000,2.000000000000000,24.773158019992341,24.326830246085336,0.000000000000000,...,-17.710941839353161,0.000000000000000,-4.499679603676096,-4.499568253197682,0.000000000000000,-0.999999999710040,-0.999999999997136,0.000000000000000,0.000000000000000,9.000000000000000
20%,3.000000000000000,51581.683857051946688,51581.683857051946688,2097.000000000000000,1020975.000000000000000,4216551.000000000000000,2.000000000000000,164.425482628458639,369.158103237661010,0.000000000000000,...,-3.744628343623937,0.000000000000000,-0.276178979847342,-0.448202606750217,0.000000000000000,-0.908754533614142,-0.990180547922742,0.000000000000000,0.000000000000000,2097.000000000000000
40%,3.000000000000000,52463.505005239218008,52463.505005239218008,3699.000000000000000,1036995.000000000000000,7906537.000000000000000,2.000000000000000,221.109751320105033,558.612295012333334,0.000000000000000,...,0.001054038304446,0.000000000000000,-0.011676194957329,-0.013512284924615,0.000000000000000,-0.103106074544829,-0.124518105254163,0.000000000000000,0.000000000000000,3699.000000000000000
50%,3.000000000000000,52837.830350991127489,52837.830350991127489,4371.000000000000000,1043715.000000000000000,9513267.000000000000000,2.000000000000000,248.005791425839476,689.903957504841173,0.000000000000000,...,3.482675846373538,0.000000000000000,0.000064176990997,0.000068296741068,0.000000000000000,-0.032343421360417,0.070449461160258,0.000000000000000,0.000000000000000,4371.000000000000000
70%,3.000000000000000,54134.101039795750694,54134.101039795750694,6117.000000000000000,1061175.000000000000000,13088902.000000000000000,2.000000000000000,624.031166410734841,886.842128104198423,0.000000000000000,...,11.537920574391816,0.000000000000000,0.096835513105000,0.161742247104170,0.000000000000000,0.079129937069842,0.783620366734324,0.000000000000000,1.000000000000000,6117.000000000000000
80%,3.000000000000000,55629.115691227903881,55629.115691227903881,7521.000000000000000,1075215.000000000000000,15537869.000000000000000,2.000000000000000,884.923197964995438,905.71812899063161

In [ ]:
#Dropping the columns that are of no use
data.drop(['type'],axis=1,inplace=True)
data.drop(['pos_z'],axis=1,inplace=True)
data.drop(['spd_z'],axis=1,inplace=True)
data.drop(['acl_z'],axis=1,inplace=True)
data.drop(['hed_z'],axis=1,inplace=True)

In [ ]:
data.shape

(820097, 17)

In [ ]:
data.head().T

,0,1,2,3,4
rcvTime,57511.049951916953432,57511.146510437574761,57511.162338829126384,57511.590730855474249,57511.641619857240585
sendTime,57511.049951916953432,57511.146510437574761,57511.162338829126384,57511.590730855474249,57511.641619857240585
sender,9891.000000000000000,9951.000000000000000,10011.000000000000000,9957.000000000000000,9963.000000000000000
senderPseudo,1098915.000000000000000,1099515.000000000000000,10100115.000000000000000,1099575.000000000000000,1099635.000000000000000
messageID,19655176.000000000000000,19655429.000000000000000,19655584.000000000000000,19656248.000000000000000,19656466.000000000000000
density_level,2,2,2,2,2
pos_x,137.286902555145161,153.904668561966304,142.981000552607952,149.931080349069163,152.753405416050725
pos_y,983.827257071713575,899.505911646706977,1038.129049878944215,899.035172728899624,906.773024725052210
spd_x,1.354661693085098,19.671425131174317,26.896996701957299,0.206780659828718,0.071913888517510
spd_y,3.904279797161307,13.272349119084248,24.137761106685371,-2.687713403544073,-0.969420576222226


In [ ]:
# dropping the column that doesn't add up any information.
data.drop(['file_name'],axis=1,inplace=True)

In [ ]:
# Sorting by receive time
data.sort_values(by=['rcvTime'], inplace=True)
data.reset_index(drop=True, inplace=True)
data.head()

,rcvTime,sendTime,sender,senderPseudo,messageID,density_level,pos_x,pos_y,spd_x,spd_y,acl_x,acl_y,hed_x,hed_y,attack,receiver_id
0,50408.529003064497374,50408.529003064497374,21.000000000000000,10215.000000000000000,16936.000000000000000,2,220.317917303648272,383.934935437669594,-0.078168163622794,0.525905153291198,-0.388921475468878,2.616637002617828,-0.142980189511322,0.989725550547881,0,27
1,50408.836187128974416,50408.836187128974416,27.000000000000000,10275.000000000000000,17018.000000000000000,2,223.030520488101104,364.802884416285281,-0.079870635180307,0.537359158655796,-0.388233666394645,2.613037040001825,-0.147020325879710,0.989133471164647,0,21
2,50409.529003193412791,50409.529003193412791,21.000000000000000,10215.000000000000000,17075.000000000000000,2,219.947049702855480,385.757716338412308,-0.421900962680975,2.838498011407711,-0.322469854764553,2.169566146413223,-0.142233817434146,0.989833087534514,0,27
3,50409.836187257868005,50409.836187257868005,27.000000000000000,10275.000000000000000,17117.000000000000000,2,222.765622516565202,366.657630560278733,-0.419577828663898,2.822925004092247,-0.360335561439105,2.425224516331284,-0.142064310031641,0.989857430044970,0,21
4,50410.529003319134063,50410.529003319134063,21.000000000000000,10215.000000000000000,17134.000000000000000,2,219.502638411099042,389.372337489329084,-0.403673484974945,3.464321108005000,0.251874296063086,-2.161571729462409,-0.110933924842951,0.993827784034506,0,27


In [ ]:
# Splitting the data into 80% as train and rest(20%) as test
total_size = len(data)
total_size
train_size = round(total_size*.8)
test_size = total_size - train_size
print('total', total_size, 'train_size', train_size, 'test_size', test_size)

total 820097 train_size 656078 test_size 164019


In [ ]:
# Populating train and test data based on the index calculated in above cell
train_data = data[0:train_size]
test_data = data[train_size:train_size+test_size]
print('train_data_size', len(train_data), 'test_data_size', len(test_data))

train_data_size 656078 test_data_size 164019


In [ ]:
# 6 features
x_train_6 = train_data[['pos_x','pos_y','spd_x','spd_y','acl_x','acl_y']]
x_test_6 = test_data[['pos_x','pos_y','spd_x','spd_y','acl_x','acl_y']]
y_train_6 = train_data[['attack']]
y_test_6 = test_data[['attack']]

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,LSTM,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import models

import keras

from keras.layers import Conv1D, MaxPool2D, Dense, Flatten, Activation

In [ ]:
print(x_train_6.shape)
print(x_test_6.shape)
print(y_train_6.shape)
print(y_test_6.shape)

(656078, 6)
(164019, 6)
(656078, 1)
(164019, 1)


In [ ]:
x_train_6 = x_train_6.values.reshape(-1, 1, 6)
x_test_6  = x_test_6.values.reshape(-1, 1, 6)
y_train_6 = y_train_6.values.reshape(-1, 1)
y_test_6 = y_test_6.values.reshape(-1, 1)


In [ ]:
print(x_train_6.shape)
print(x_test_6.shape)
print(y_train_6.shape)
print(y_test_6.shape)

(656078, 1, 6)
(164019, 1, 6)
(656078, 1)
(164019, 1)


In [ ]:
model = Sequential()

model.add((Conv1D(filters=5, kernel_size=2, activation='relu', padding='same')))
model.add(LSTM(256,activation='relu',return_sequences=True))
model.add(LSTM(256,activation='relu',return_sequences=True))
model.add(LSTM(256,activation='relu',return_sequences=True))
model.add(LSTM(256,activation='relu'))
model.add(Dense(2,activation='softmax'))


model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(x_train_6,y_train_6,epochs=5,validation_data=(x_test_6,y_test_6),steps_per_epoch=len(x_train_6)/256)



Epoch 1/5
2562/2562 [==============================] - 56s 17ms/step - loss: 0.1740 - accuracy: 0.9331 - val_loss: 0.1243 - val_accuracy: 0.9587
Epoch 2/5
2562/2562 [==============================] - 41s 16ms/step - loss: 0.1334 - accuracy: 0.9524 - val_loss: 0.1217 - val_accuracy: 0.9584
Epoch 3/5
2562/2562 [==============================] - 41s 16ms/step - loss: 0.1286 - accuracy: 0.9537 - val_loss: 0.1234 - val_accuracy: 0.9640
Epoch 4/5
2562/2562 [==============================] - 41s 16ms/step - loss: 0.1256 - accuracy: 0.9544 - val_loss: 0.1213 - val_accuracy: 0.9652
Epoch 5/5
2562/2562 [==============================] - 41s 16ms/step - loss: 0.1231 - accuracy: 0.9549 - val_loss: 0.1310 - val_accuracy: 0.9553


In [ ]:
ypred_x_train=model.predict(x_train_6)
ypred_x_train.argmax(axis=1).tolist()

[0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,


In [ ]:
y_train_6.tolist()

[[0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [1],
 [1],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [1],
 [1],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [1],
 [1],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [1],
 [1],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [1],
 [1],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [1],
 [1],
 [1],
 [0],
 [0]

In [ ]:
ypred=model.predict(x_test_6)
ypred=ypred.argmax(axis=1)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
# Finding accuracy, precision, recall and confusion matrix
print(accuracy_score(y_test_6,ypred))
print(classification_report(y_test_6,ypred))

0.9553100555423457
              precision    recall  f1-score   support

           0       0.96      0.98      0.97    114799
           1       0.94      0.91      0.92     49220

    accuracy                           0.96    164019
   macro avg       0.95      0.94      0.95    164019
weighted avg       0.96      0.96      0.96    164019



In [ ]:
print(confusion_matrix(y_test_6,ypred))

[[112144   2655]
 [  4675  44545]]
